In [1]:
import pandas as pd

In [2]:
file_type = 'thread'
field_name = 'title'

In [3]:
df = pd.read_csv(f'{file_type}_with_yap.csv')

In [4]:
df_err = df.loc[df[f'{field_name}_dict'].str.contains('error')]

In [5]:
ratio = float(df_err.shape[0]) / df.shape[0]
print(f'ratio of errors {ratio}')

ratio of errors 0.07281242494596109


In [6]:
df = df.loc[~df[f'{field_name}_dict'].str.contains('error')]

In [7]:
def get_tokenized_from_md_lattice(md_lattice_text,use_lemma=True,keep_only_NN=False):
    tokens = list()
    token_blocks = md_lattice_text.split('\n')
    for token_block in token_blocks:
        token_block_parts = token_block.split('\t')
        if len(token_block_parts) > 2:
            if keep_only_NN and token_block_parts[4] != 'NN':
                continue
            if use_lemma:
                tokens.append(token_block_parts[2])
            else:
                tokens.append(token_block_parts[3])
    return tokens

In [8]:
df['token_set'] = df[f'{field_name}_dict'].apply(lambda x: get_tokenized_from_md_lattice(eval(x)['md_lattice'], False, True))

In [9]:
df.head()

,Unnamed: 0,Unnamed: 0.1,thread_id,title,type,title_dict,token_set
0,0,0,20216916,"עמותות בצל משבר הקורונה - בואו להתעניין, להיעז...",NaN,{'ma_lattice': '0\t1\tעמותות\tעמותה\tNNT\tNNT\...,"[עמותה, קורונה, התעניין,, עזור.]"
1,1,0,20289621,שיבוץ לתפקיד שאני לא רוצה - FXP,NaN,{'ma_lattice': '0\t1\tשיבוץ\tשיבוץ\tNNT\tNNT\t...,"[שיבוץ, תפקיד]"
2,2,0,19881195,דרושים מנהלים לפורום צבא וביטחון! - FXP,NaN,{'ma_lattice': '0\t1\tדרושים\tדרש\tBN\tBN\tgen...,"[מנהל, צבא, יטחון!]"
3,3,0,19801135,השימושון של פורום צבא וביטחון - FXP,NaN,{'ma_lattice': '0\t1\tה\tה\tDEF\tDEF\t_\t1\n0\...,"[שימושון, צבא, ביטחון]"
4,4,0,20246418,יום הזיכרון בFXP - פורום הנצחה והזדהות - FXP,NaN,{'ma_lattice': '0\t1\tיום\tיום\tNNT\tNNT\tgen=...,"[זיכרון, הנצחה, הזדהות]"


In [10]:
corpus = df['token_set'].tolist()

In [11]:
import pickle
with open(f'{file_type}_{field_name}_token_lists.pickle', 'wb') as handle:
    pickle.dump(corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)